### Install WEb3

In [2]:
pip install web3

Note: you may need to restart the kernel to use updated packages.


### Connecting with the local host using Ganache

In [2]:
from web3 import Web3
import solcx

# Solidity source code
storage_source_code = '''
// SPDX-License-Identifier: GPL-3.0

pragma solidity >=0.8.2 <0.9.0;

/**
 * @title Storage
 * @dev Store & retrieve value in a variable
 * @custom:dev-run-script ./scripts/deploy_with_ethers.ts
 */
contract Storage {

    uint256 number;

    /**
     * @dev Store value in variable
     * @param num value to store
     */

    
    function store(uint256 num) public {
        number = num;
    }

    /**
     * @dev Return value 
     * @return value of 'number'
     */
    function retrieve() public view returns (uint256){
        return number;
    }
}
'''

# Install and set Solidity compiler version
solcx.install_solc('0.8.3')
solcx.set_solc_version('0.8.3')

# Compile the Solidity contract
compiled_sol = solcx.compile_source(storage_source_code)
contract_interface = compiled_sol['<stdin>:Storage']

# Extract ABI and Bytecode
abi = contract_interface['abi']
bytecode = contract_interface['bin']

# Connect to Ganache
w3 = Web3(Web3.HTTPProvider('http://127.0.0.1:7545'))

# Check connection
if w3.is_connected():
    print("Connected to Ganache")
else:
    print("Failed to connect to Ganache")

# Use an account from Ganache
account_address = w3.eth.accounts[0]
private_key = '0x9e338fe1683ba403f9a763880423d74c9d95453077a34c31aa3c72b6bc8f4162'

# Get the latest transaction nonce for the account
nonce = w3.eth.get_transaction_count(account_address)

# Create the contract object
StorageContract = w3.eth.contract(abi=abi, bytecode=bytecode)

# Build the transaction
transaction = StorageContract.constructor().build_transaction({
    'chainId': 1337,  # Chain ID for Ganache
    'gas': 2000000,   # Gas limit
    'gasPrice': w3.eth.gas_price,  # Gas price
    'nonce': nonce,   # Nonce
})

# Sign the transaction with your private key
signed_txn = w3.eth.account.sign_transaction(transaction, private_key=private_key)

# Send the signed transaction
tx_hash = w3.eth.send_raw_transaction(signed_txn.raw_transaction)

# Wait for the transaction receipt
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

# Print the transaction receipt and contract address
print(f"Contract deployed at address: {tx_receipt.contractAddress}")

# Interact with the contract
deployed_contract = w3.eth.contract(address=tx_receipt.contractAddress, abi=abi)

# Store a value in the contract
store_tx = deployed_contract.functions.store(42).build_transaction({
    'chainId': 1337,
    'gas': 200000,
    'gasPrice': w3.eth.gas_price,
    'nonce': w3.eth.get_transaction_count(account_address),
})
signed_store_tx = w3.eth.account.sign_transaction(store_tx, private_key=private_key)
store_tx_hash = w3.eth.send_raw_transaction(signed_store_tx.raw_transaction)
print(f"Stored value transaction hash: {store_tx_hash.hex()}")

# Retrieve the stored value
value = deployed_contract.functions.retrieve().call()
print(f"Stored value: {value}")


Connected to Ganache
Contract deployed at address: 0xF0D0021c405f6DBB50195Ad6E4bfBaA5282D4277
Stored value transaction hash: f1d11e4509bf992e80354b9a543ba0abc726268bb8e1c4ccc1d2b5856aee0d30
Stored value: 42
